<a href="https://colab.research.google.com/github/VGGatGitHub/Summer2021/blob/main/D-Wave_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook experimenting how to use DOcplex and IBM Quantum.

# 0. Install libraries & clone repo

In [ ]:
#https://www.youtube.com/watch?v=bErs0dxC1aY&list=PLPvKnT7dgEsuhec_yW9oxcZ6gg9SfRFFV
#https://docs.ocean.dwavesys.com/en/latest/overview/install.html#installoceansoftware

#!pip install dwave-ocean-sdk
!pip show dwave-ocean-sdk

In [2]:
#!dwave setup #API endpoint URL: https://cloud.dwavesys.com/sapi/

In [ ]:
import sys

if hasattr(sys, 'real_prefix'):
    #we are in a virtual env.
    !pip install cplex && pip install docplex
else:
    !pip install --user cplex && pip install --user docplex

print()
!pip show cplex
print()
!pip show docplex

#it my be useful to retstart you runtime

In [ ]:
!pip install qiskit
!pip show qiskit

# !pip install qiskit-optimization
#!pip show qiskit-optimization

#it my be useful to retstart you runtime

In [ ]:
#it my be useful to retstart you runtime

# clone repo
!git clone https://github.com/VGGatGitHub/Summer2021.git

#**Possibly need to restart runtime here.**

# 1. Load Data

In [6]:
#next run the model.py in the folder introScenario
!pwd
%cd ./Summer2021/introScenario/
#./Summer2021/intermediateScenario
#./Summer2021/introScenario/
#%ls


/content
/content/Summer2021/introScenario


In [7]:
import pandas as pd

# supply + costs
supply_file = './plants.csv'

# demand
demand_file = './customerDemand.csv'

# for now load example data (plants.csv and customerDemand.csv under introScenario folder)
s_df = pd.read_csv(supply_file)
d_df = pd.read_csv(demand_file)

In [8]:
d_df

,Product,Demand
0,handSanitizer,100
1,mask,120


In [9]:
s_df

,Plants,Cost,Capacity,Product
0,1,3,40,mask
1,2,2,30,mask
2,3,1,30,handSanitizer
3,4,3,100,handSanitizer
4,5,2,60,mask
5,6,1,45,mask


In [10]:
inputs={'customerDemand':d_df,'plants':s_df}

# 2. Solve the DO problem via DOcplex

In [11]:
#If you get an error you may have to restart the runtime
import cplex
import docplex.mp
from docplex.mp.model import Model

In [12]:
global output_lock
#with output_lock:

try:  
  if output_lock:
    print(output_lock)
except:
  output_lock=dict({'output_lock':{}})

try: 
  if outputs:
    print(outputs)
except:
  outputs=dict()


In [13]:
exec(open("./model.py").read())

* building wado model
model building done in 0.15670061111450195 secs
* running wado model
Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               120
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 20 rows and 12 columns.
All rows and columns eliminated.
Presolve time = 0.01 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.02 sec. (0.01 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.02 sec. (0.01 ticks)
Solve status: JobSolveStatus.OPTIMAL_SOLUTION
* model docplex_model1 solved with objective = -105680.000
*  KPI: total plants allocations                  = 112640.000
*  KPI: total Cost of plants over all all

In [14]:
outputs


{'plantsAllocation':    plants  ...  plants minValueAllocationForAssignment
 0       1  ...                                       0
 1       2  ...                                       0
 2       3  ...                                       0
 3       4  ...                                       0
 4       5  ...                                       0
 5       6  ...                                       0
 
 [6 rows x 7 columns]}

In [15]:
outputs['plantsAllocation']

,plants,plants allocation decision,plants selection decision,plants Capacity,plants Product,plants Cost,plants minValueAllocationForAssignment
0,1,0.0,1.0,40,mask,3,0
1,2,15.0,1.0,30,mask,2,0
2,3,30.0,1.0,30,handSanitizer,1,0
3,4,70.0,1.0,100,handSanitizer,3,0
4,5,60.0,1.0,60,mask,2,0
5,6,45.0,1.0,45,mask,1,0


In [16]:
model

docplex.mp.Model['docplex_model1']

In [17]:
solution=model.solve()
print(solution)
solution

solution for: docplex_model1
objective: -105680
x2=15
x3=30
x4=70
x5=60
x6=45
x7=1
x8=1
x9=1
x10=1
x11=1
x12=1



docplex.mp.solution.SolveSolution(obj=-105680,values={x2:15,x3:30,x4:70,..

In [18]:
cost = solution.get_objective_value()
x = solution.get_all_values()
x = np.array(x)
print(cost,x)

-105680.0 [ 0. 15. 30. 70. 60. 45.  1.  1.  1.  1.  1.  1.]


# 3. Solve the problem using D-Wave

In [19]:
# take a look at https://quantum-computing.ibm.com/lab/docs/iql/optimization

#from qiskit_optimization import QuadraticProgram
#from qiskit_optimization.translators import from_docplex_mp

from qiskit.optimization import QuadraticProgram

# load from a Docplex model
mod = QuadraticProgram()
mod.from_docplex(model)
print(mod.export_as_lp_string())

/usr/local/lib/python3.7/dist-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/usr/local/lib/python3.7/dist-packages/qiskit/optimization/__init__.py:92: DeprecationWarning: The package qiskit.optimization is deprecated. It was moved/refactored to qiskit_optimization (pip install qiskit-optimization). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('optimization', 'qiskit_optimization', 'qiskit-optimization')


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: - 464 x0 - 480 x1 - 496 x2 - 464 x3 - 480 x4 - 496 x5
Subject To
 _L_EXPR_1: x0 - 40 x6 <= 0
 _L_EXPR_2: x1 - 30 x7 <= 0
 _L_EXPR_3: x2 - 30 x8 <= 0
 _L_EXPR_4: x3 - 100 x9 <= 0
 _L_EXPR_5: x4 - 60 x10 <= 0
 _L_EXPR_6: x5 - 45 x11 <= 0
 _L_EXPR_7: x0 >= 0
 _L_EXPR_8: x1 >= 0
 _L_EXPR_9: x2 >= 0
 _L_EXPR_10: x3 >= 0
 _L_EXPR_11: x4 >= 0
 _L_EXPR_12: x5 >= 0
 _L_EXPR_13: x0 <= 40
 _L_EXPR_14: x1 <= 30
 _L_EXPR_15: x2 <= 30
 _L_EXPR_16: x3 <= 100
 _L_EXPR_17: x4 <= 60
 _L_EXPR_18: x5 <= 45
 _L_EXPR_19: x2 + x3 <= 100
 _L_EXPR_20: x0 + x1 + x4 + x5 <= 120

Bounds
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1
 0 <= x9 <= 1
 0 <= x10 <= 1
 0 <= x11 <= 1

Binaries
 x6 x7 x8 x9 x10 x11

Generals
 x0 x1 x2 x3 x4 x5
End



In [20]:
print('constant:\t\t\t', mod.objective.constant)

print('linear dict:\t\t\t', mod.objective.linear.to_dict())
print('linear array:\t\t\t', mod.objective.linear.to_array())
print('linear array as sparse matrix:\n', mod.objective.linear.coefficients, '\n')

print('quadratic dict w/ index:\t', mod.objective.quadratic.to_dict())
print('quadratic dict w/ name:\t\t', mod.objective.quadratic.to_dict(use_name=True))
print('symmetric quadratic dict w/ name:\t', mod.objective.quadratic.to_dict(use_name=True, symmetric=True))
print('quadratic matrix:\n', mod.objective.quadratic.to_array(),'\n')
print('symmetric quadratic matrix:\n', mod.objective.quadratic.to_array(symmetric=True),'\n')
print('quadratic matrix as sparse matrix:\n', mod.objective.quadratic.coefficients)


constant:			 0.0
linear dict:			 {0: -464.0, 1: -480.0, 2: -496.0, 3: -464.0, 4: -480.0, 5: -496.0}
linear array:			 [-464. -480. -496. -464. -480. -496.    0.    0.    0.    0.    0.    0.]
linear array as sparse matrix:
   (0, 0)	-464.0
  (0, 1)	-480.0
  (0, 2)	-496.0
  (0, 3)	-464.0
  (0, 4)	-480.0
  (0, 5)	-496.0 

quadratic dict w/ index:	 {}
quadratic dict w/ name:		 {}
symmetric quadratic dict w/ name:	 {}
quadratic matrix:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 

symmetric quadratic matrix:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.

In [24]:
import dimod


# Extract qubo data
Q = mod.objective.quadratic.to_dict(use_name=True)
g = mod.objective.linear.to_dict(use_name=True)
c = mod.objective.constant

# Build BQM
bqm=dimod.BQM(g, Q, c, dimod.BINARY)

In [ ]:
from dwave.system import LeapHybridSampler
from dwave.system import DWaveSampler, EmbeddingComposite

# Solve using LeapHybridSampler
sampler = LeapHybridSampler()
bqm.result = sampler.sample(bqm)

# Extract solution
result_dict = bqm.result.first.sample
extract_solution(result_dict)